In [1]:
from DataProcessing import DataProcessing
from Car import Car
from Cba import Cba
from Validation import Validation

## 原版 CBA


In [2]:
data_procesing = DataProcessing("car.data")
data_procesing.read_data()
data_procesing.process()
data = data_procesing.encoded_data
train_data, test_data = data_procesing.split_train_test_data()
MIN_SUPPORT = 0.1
MIN_CONFIDENCE = 0.7
car = Car(train_data, MIN_SUPPORT, MIN_CONFIDENCE)
car.generate_frequent()
car.sort_rule(1)
cba = Cba(train_data, car.rule)
cba.cover()
rules = cba.final_rules_
default = cba.default
test = Validation(test_data, rules, default, 'single')

3=0 => class=2 (support=0.3379160636758321, confidence=1.0, lift=1.4188911704312117)
5=1 => class=2 (support=0.33646888567293776, confidence=1.0, lift=1.4188911704312117)
0=3 => class=2 (support=0.20477568740955138, confidence=0.827485380116959, lift=1.174111699508868)
1=3 => class=2 (support=0.20477568740955138, confidence=0.827485380116959, lift=1.174111699508868)
4=2 => class=2 (support=0.25253256150506515, confidence=0.7842696629213484, lift=1.1127932999561638)
2=0 => class=2 (support=0.18885672937771347, confidence=0.7721893491124261, lift=1.0956526493566456)
0=0 => class=2 (support=0.19681620839363242, confidence=0.7555555555555556, lift=1.0720511065480267)
3
Macro F1 Score: 0.35839441136051303
Accuracy: 0.6676300578034682


## WCBA 隨機森林


In [3]:
data_procesing.get_feature_importances()
feature_importances = data_procesing.feature_importances
MIN_SUPPORT = 0.1
MIN_CONFIDENCE = 0.7
car = Car(data=train_data, min_support=MIN_SUPPORT,
          min_confidence=MIN_CONFIDENCE, weights=feature_importances)
car.generate_frequent()
car.sort_rule(3, True)
cba = Cba(train_data, car.rule)
cba.cover()
rules = cba.final_rules_
default = cba.default
test = Validation(test_data, rules, default, 'group')

2=0 => class=2 (support=0.18885672937771347, weight_supprot=0.19981041968162086, confidence=0.7721893491124261, lift=1.0956526493566456)
1=0 => class=2 (support=0.17945007235890015, weight_supprot=0.20816208393632415, confidence=0.7315634218289085, lift=1.0380088798434821)
0=0 => class=2 (support=0.19681620839363242, weight_supprot=0.23519536903039076, confidence=0.7555555555555556, lift=1.0720511065480267)
1=3 => class=2 (support=0.20477568740955138, weight_supprot=0.23753979739507958, confidence=0.827485380116959, lift=1.174111699508868)
0=3 => class=2 (support=0.20477568740955138, weight_supprot=0.2447069464544139, confidence=0.827485380116959, lift=1.174111699508868)
4=2 => class=2 (support=0.25253256150506515, weight_supprot=0.27399782923299565, confidence=0.7842696629213484, lift=1.1127932999561638)
3=0 => class=2 (support=0.3379160636758321, weight_supprot=0.41834008683068014, confidence=1.0, lift=1.4188911704312117)
Macro F1 Score: 0.2834211329974042
Accuracy: 0.566473988439306